# [FootShirt.com](https://www.footshirt.com)
![alt text](https://www.footshirt.com/assets/logo/footshirtlogo-small.png "FootShirt.com")

[FootShirt.com](https://www.footshirt.com) is the site for soccer fans that enjoy collecting jerseys and/or are passionate by the history and values communicated by soccer shirts. 

Managing your jerseys collection can not be easier. Easily add your jerseys to your digital closet. Browse through the available jerseys or add your own. 
Add the jerseys you plan to buy or receive as a gift to your Wishlist that you can share with friends and family. 

Read our blog that features articles about soccer shirts. Do you think you know everything about soccer jerseys? Try our quizzes and compete with other 
fans around the world to be on the top of the leaderboard.



# Introduction

The goal of this notebook is to demonstrate the use of data visualization techniques and libraries while using hopefully a fun dataset. We will focus on Soccer Jersey Dominant Color in this notebook and create many 3D scatter plots.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ochadenat/jerseycolordetection/master?filepath=jerseyColorDetection.ipynb)

# Dataset

## Home, Away and Third Jerseys
For a very long time, teams had only two jerseys referenced as Home and Away. These two jersey types had very distinct colors in case two teams having the same main color where playing against each other (think Manchester United versus Liverpool Red Jerseys). It's worth mentioning that many teams have still a Away jersey with white as the dominant color, likely an heritage from the black and white TV era.

## Records
The dataset has 3,500 records (i.e. jersey) covering x teams (X Clubs and Y National Teams). Some teams might have only 1 jersey in the dataset while other popular teams such as Manchester United will have X jerseys. In addition, the recent period 2015-2019 is more represented compared to other periods : C jerseys for 2010-2014, D jerseys for 2005-2009 and E for pre-2005.
This will introduce a bias toward popular teams, the recent past, the Home jersey in some analysis.Goalkeeper jerseys have been excluded from this dataset

## Fields Description
| Field                 | Description   | Example/Value
| --------------------- | --- | ---
| **id**                | jersey unique id                                    | 1, 2, 3
| **team_id**           | team unique id                                      | 1, 2, 3
| **teamname**          | team name                                           | Boca Juniors
| **teamtype**          | team type                                           | Club, National   
| **teamcountry**       | team country                                        | Argentina    
| **image_caption**     | concatenation of team name, season, and jersey type | Boca Juniors 2018-2019 Home
| **season**            | season                                              | 2018-2019
| **homeawaythird**     | jersey type                                         | Home, Away, Third, Goalkeeper  
| **image_color**       | jersey dominant color                               | rgb(50,72,81)
| **image_r**           | jersey dominant color red component                 | Number between 0 and 255, e.g. 50  
| **image_g**           | jersey dominant color green component               | Number between 0 and 255, e.g. 72
| **image_b**           | jersey dominant color blue component                | Number between 0 and 255, e.g. 81
| **image_content_url** | ?                                                   | ?
| **image_id**          | ?                                                   | 1
| **brand_id**          | kit supplier unique id                              | 1, 2, 3
| **brandname**         | kit supplier                                        | Adidas, Nike, Puma
| **urlname**           | ?                                                   |        

Each jersey is assigned one (and only one) dominant color. We will use the RGB Color model in which each parameter (Red, Green, and Blue) defines the intensity of the color as an integer between 0 and 255.A future analysis will handle several dominant colors to take into account shirts such as Juventus Turin jerseys where both black and white can be considered as dominant colors.

### Import

In [ ]:
import struct
from PIL import Image
import numpy as np
import webcolors as wb
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns; sns.set()  # for plot styling
import scipy
from scipy.cluster.vq import vq, kmeans, whiten
from  scipy.spatial.distance  import cdist
from math import sqrt
from matplotlib import rcParams
# from matplotlib import style
import matplotlib.gridspec as gridspec


from IPython.display import HTML, display


import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

# Configure Matplotlib
# to render the figure in a notebook
%matplotlib inline
# Set Matplotlib default padding for title label to 20 points
rcParams["axes.titlepad"] = 20
# Set Matplotlib default padding for axis label to 5 points
rcParams["axes.labelpad"] = 5
# Set Matplotlib default background color to white
rcParams["figure.facecolor"] = "w"



## Define Dataset


In [ ]:
JERSEYS_COLORS_FILENAME = "jerseys_colors.csv"


In [ ]:
#Display a HTML table with jersey name, picture and dominant color
def display_jersey_table(df):
    
    print (df.head(5))
    data = df.values
    html_str = "<table><tr><th>Jersey</th><th>Picture</th><th>Color</th></tr>"
    for x in range(0, data.shape[0]):
        html_str = html_str + "<td>{}</td>".format(data[x][0])
        html_str = html_str + "<td><img src=\"{}\"  height=42 width=42></td>".format(data[x][1])
        html_str = html_str + "<td style=\"background-color:{}\"></td></tr>".format(data[x][2])
    html_str = html_str + "</table>"
    
    display(HTML(html_str))


# Chart Colors

Break Down by year/period


Break Down by Country


# Football Shirts Dominant Colors
The chart below show dominant colors for the entire FootShirt.com database, i.e about 3,000 jerseys. It"s worth noticing 
that very few teams play in green

In [ ]:
df_colors = pd.read_csv(JERSEYS_COLORS_FILENAME, sep=";",usecols  =["image_r", "image_g", "image_b"])
X = df_colors[["image_r", "image_g", "image_b"]].values
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=20, edgecolors="none")
ax.set_xlabel("Red", fontsize=14, fontweight="bold", color="r")
ax.set_ylabel("Green",  fontsize=14, fontweight="bold", color="g")
ax.set_zlabel("Blue",  fontsize=14, fontweight="bold", color="b", rotation=90)
ax.set_title( "Football Shirts Dominant Colors", fontsize=14, fontweight="bold")
plt.show()


# Football Shirts Dominant Colors for the 2018 Fifa World Cup Russia

The 2018 Fifa World Cup Russia was featuring 32 teams, each team had a Home Jersey and a Away Jersey (so the dataset has 64 jerseys, each having a dmoninant color

In [ ]:
# Define the scope
teamnames = ["Argentina", "Australia", "Belgium", "Brazil", "Colombia", "Costa Rica", "Croatia", "Denmark", "Egypt", 
             "England", "France", "Germany", "Iceland", "Iran", "Japan", "South Korea", "Mexico", "Morocco", 
             "Nigeria", "Panama", "Peru", "Poland", "Portugal", "Russia", "Saudi Arabia", "Senegal", "Serbia", 
             "Spain", "Sweden", "Switzerland", "Tunisia", "Uruguay"]
season = "2018"
jersey_types = ["Home", "Away"]
# Load Dataset
df_colors = pd.read_csv(JERSEYS_COLORS_FILENAME, sep=";",usecols  =["teamname", "season", "homeawaythird", "image_r", "image_g", "image_b","teamname"])

fig = plt.figure(figsize=(24,6))
fig.subplots_adjust(hspace=0.1, wspace=0.01)

df_colors_type = df_colors [(df_colors["teamname"].isin(teamnames)) & (df_colors["homeawaythird"].isin(jersey_types)) &
    (df_colors["season"] == "2018")]
X = df_colors_type[["image_r", "image_g", "image_b"]].values
ax = fig.add_subplot(131, projection="3d")
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=200, linewidths=1, edgecolors="k")
ax.set_xlim3d(0,300)
ax.set_ylim3d(0,300)
ax.set_zlim3d(0,300)
ax.set_xlabel("Red", fontsize=12, fontweight="bold", color="r")
ax.set_ylabel("Green",  fontsize=12, fontweight="bold", color="g")
ax.set_zlabel("Blue",  fontsize=12, fontweight="bold", color="b", rotation=90)
title = "Football Shirt Dominant Colors for World Cup 2018\n (sample size: {})".format(len(df_colors_type))
ax.set_title( title, fontsize=12, fontweight="bold",  pad = 0)
 
i = 1
for jersey_type in jersey_types:
    df_colors_type = df_colors [(df_colors["teamname"].isin(teamnames)) & (df_colors["homeawaythird"] == jersey_type) &
    (df_colors["season"] == "2018")]
    X = df_colors_type[["image_r", "image_g", "image_b"]].values
    ax = fig.add_subplot(1,len (jersey_types)+1, i + 1, projection="3d")
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=200, linewidths=1, edgecolors="k")
    ax.set_xlim3d(0,300)
    ax.set_ylim3d(0,300)
    ax.set_zlim3d(0,300)
    ax.set_xlabel("Red", fontsize=12, fontweight="bold", color="r")
    ax.set_ylabel("Green",  fontsize=12, fontweight="bold", color="g")
    ax.set_zlabel("Blue",  fontsize=12, fontweight="bold", color="b", rotation=90)
    title = "Football Shirt Dominant Colors for World Cup 2018 - {} jerseys\n (sample size: {}".format(jersey_type,len(df_colors_type))
    ax.set_title(title , fontsize=12, fontweight="bold", pad = 0)
    i += 1




# Liverpool

In [ ]:
df_colors = pd.read_csv(JERSEYS_COLORS_FILENAME, sep=";",usecols  =["image_r", "image_g", "image_b","teamname"])
df_colors = df_colors [df_colors["teamname"]=="Liverpool"]
X = df_colors[["image_r", "image_g", "image_b"]].values
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=50, edgecolors="none")


ax.set_xlabel("Red", fontsize=14, fontweight="bold", color="r")
ax.set_ylabel("Green",  fontsize=14, fontweight="bold", color="g")
ax.set_zlabel("Blue",  fontsize=14, fontweight="bold", color="b", rotation=90)
ax.set_title( "Football Shirt Dominant Colors", fontsize=14, fontweight="bold", pad=20)
plt.show()



## Liverpool Home/Away/Third Jersey

In [ ]:
teamname = "Liverpool"
# Define the Jersey Type that we will take into account
jersey_types = ["Home", "Away", "Third"]
# Load Dataset
df_colors = pd.read_csv(JERSEYS_COLORS_FILENAME, sep=";",usecols  =["image_r", "image_g", "image_b","teamname", "homeawaythird"])

fig = plt.figure(figsize=(24,6))
fig.subplots_adjust(hspace=0.1, wspace=0.01)
  
i = 1
for jersey_type in jersey_types:
    df_colors_type = df_colors [(df_colors["teamname"]== teamname) & (df_colors["homeawaythird"] == jersey_type)]
    X = df_colors_type[["image_r", "image_g", "image_b"]].values
    ax = fig.add_subplot(1, len (jersey_types), i, projection="3d")
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=50, edgecolors="none")
    ax.set_xlim3d(0,300)
    ax.set_ylim3d(0,300)
    ax.set_zlim3d(0,300)
    ax.set_xlabel("Red", fontsize=12, fontweight="bold", color="r")
    ax.set_ylabel("Green",  fontsize=12, fontweight="bold", color="g")
    ax.set_zlabel("Blue",  fontsize=12, fontweight="bold", color="b", rotation=90)
    ax.set_title( "Football Shirt Dominant Colors for "+ teamname + " "+  jersey_type + " jerseys", fontsize=12, fontweight="bold", pad = 10)
    i += 1



## Comparison between two teams

In [ ]:
teamnames = ["Manchester United", "Arsenal"]
# Define the Jersey Type that we will take into account
jersey_type = "Home"

# Load Dataset
df_colors = pd.read_csv(JERSEYS_COLORS_FILENAME, sep=";", index_col=False,
    usecols  =["image_caption", "image_content_url", "image_color", "image_r", "image_g", "image_b","teamname", "homeawaythird"])

fig = plt.figure(figsize=(24,12))
  
df_colors_team1 = df_colors [(df_colors["teamname"]== teamnames[0]) & (df_colors["homeawaythird"] == jersey_type)]
X = df_colors_team1[["image_r", "image_g", "image_b"]].values
df_colors_team2 = df_colors [(df_colors["teamname"]== teamnames[1]) & (df_colors["homeawaythird"] == jersey_type)]
Y = df_colors_team2[["image_r", "image_g", "image_b"]].values
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=X/255.0, s=100, marker ="o", linewidths=2, edgecolors="k")
ax.scatter(Y[:, 0], Y[:, 1], Y[:, 2], c=Y/255.0, s=100, marker ="D", linewidths=2, edgecolors="k")
ax.set_xlabel("Red", fontsize=12, fontweight="bold", color="r")
ax.set_ylabel("Green",  fontsize=12, fontweight="bold", color="g")
ax.set_zlabel("Blue",  fontsize=12, fontweight="bold", color="b", rotation=90)
ax.set_title( "Football Shirt Dominant Colors comparison between "+ teamnames[0] + " and " + teamnames[1] + " " +  jersey_type + " jerseys", fontsize=12, fontweight="bold", pad = 10)




In [ ]:
display_jersey_table(df_colors_team1[["image_caption", "image_content_url", "image_color"]])

Color Picker
https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
widgets.ColorPicker(
    concise=False,
    description="Pick a color",
    value="blue",
    disabled=False
)


Country: careful because some teams have more jersey...
Season: perhaps
Country/Season/Type?
2000-2005, 2006-2010, 2011-2015, 2016-2019





Plotly test


In [ ]:
print("Implement Plotly")
